### Imports

In [21]:
%pip install -r dependencies.txt

  Using cached aiofiles-22.1.0-py3-none-any.whl (14 kB)
  Using cached aiosqlite-0.18.0-py3-none-any.whl (15 kB)
  Using cached anyio-3.6.2-py3-none-any.whl (80 kB)
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
  Using cached argon2_cffi_bindings-21.2.0-cp38-abi3-macosx_10_9_universal2.whl (53 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached Babel-2.12.1-py3-none-any.whl (10.1 MB)
  Using cached bardapi-0.1.38-py3-none-any.whl.metadata (21 kB)
  Using cached beautifulsoup4-4.11.2-py3-none-any.whl (129 kB)
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
  Using cached browser_cookie3-0.19.1-py3-none-any.whl.metadata (632 bytes)
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached cffi-1.15.1-cp310-cp310-macosx_10_9_x86_64.whl (179 kB)
  Using cached charset_normalizer-3.1.0-cp310-cp310-macosx_10_9_x86_64.whl

In [22]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

<module 'helper' from '/Users/mike/Documents/School/Fa23/Clinic/code/helper.py'>

### CONSTANTS

In [23]:
# USER
USER = "RYAN"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

In [24]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

1: OLS
2: LASSO
3: XGBOOST


### Read data

In [25]:
train_start, train_end  = helper.get_train_from_testday(TEST_START_DATE)
train_start, train_end

('20140501', '20150501')

In [26]:
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", "relvol_nt_0"]
train_df = helper.get_df(train_start, train_end, x_cols, FILE_PATH)
train_df2 = pd.read_csv(consts.CLEANED_DATA_PATH)

### Interaction Terms

In [27]:
# interactingTerms = [["relvol_nt_0", col] for col in x_cols[:-1]]
# interactingTerms

In [28]:
# interactingTerms_df = helper.get_df_with_interaction_terms(train_df, interactingTerms)

In [29]:
# interactingTerms_df.head()

In [30]:
# rosy: making my own interacting terms

# col_pairs = [
#     ['relvol_nt_0', 'rrirpnxm_nt_0'],
#     ["relvol_lst15_0", "rrirpnxm_lst15_0"],
#     ["relvol_toxhr_0", "rrirpnxm_toxhr_0"],
#     ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
# ]
# interaction_terms_train_df = helper.get_df_with_interaction_terms(train_df, col_pairs)
# interaction_terms_test_df = helper.get_df_with_interaction_terms(test_df, col_pairs)

### Transform the data if needed

#### Ordinary Least Squares

In [31]:
importlib.reload(helper)

ols_regression_model = helper.Regression('OLS')
weights = helper.get_weights(train_df)
model_attributes = ols_regression_model.train(train_df, RESPONSE_NAME, weights)
model_attributes

array([ 3.53454109e-02, -1.70054227e-03, -1.12804130e-03, -2.14339701e-03,
        5.69328897e-05,  4.41713229e-05])

In [32]:
ols_regression_model.evaluateModel(TEST_START_DATE, TEST_END_DATE, x_cols, data_path=FILE_PATH)

Average Weighted Correlation: 0.5169264253443359
Average Mean Return: 0.00019003842888847753
Average weighted Scale Factor: 0.9836675308378866


(0.5169264253443359, 0.00019003842888847753, 0.9836675308378866)

#### OLS w/ Interacting Terms

In [33]:
ols_interacting_model = helper.Regression('OLS')
#ols_interacting_model.execute(interactingTerms_df, RESPONSE_NAME, interactingTerms_df)
# ols_interacting_model.get_metric()

#### LASSO

In [34]:
lasso_model = helper.Regression('LASSO')
weights = helper.get_weights(train_df)

In [35]:
importlib.reload(helper)
lasso_model.train(train_df, RESPONSE_NAME, weights)
lasso_model.evaluateModel(TEST_START_DATE, TEST_END_DATE, x_cols, FILE_PATH)

Average Weighted Correlation: 0.5165657158406268
Average Mean Return: 0.0001706219546206434
Average weighted Scale Factor: 1.0569215795138687


(0.5165657158406268, 0.0001706219546206434, 1.0569215795138687)

### XGBoost



In [36]:
xgb_model = helper.Regression('XGboost')
weights = helper.get_weights(train_df)
xgb_model.train(train_df, RESPONSE_NAME, weights)
xgb_model.evaluateModel(TEST_START_DATE, TEST_END_DATE, x_cols, FILE_PATH)

/usr/local/lib/python3.10/site-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Average Weighted Correlation: 0.5169264251681439
Average Mean Return: 0.00019003842888847753
Average weighted Scale Factor: 0.9836675524711609


(0.5169264251681439, 0.00019003842888847753, 0.98366755)